This notebook is used to scrap the episodes of the podcast La Historia Por Concostrina.

The RSS feed is available at https://fapi-top.prisasd.com/podcast/playser/todo_concostrina/itunestfp/podcast.xml

The episodes are in the <item> tag.

The title is in the <title> tag.

The description is in the <description> tag.

The pubDate is in the <pubDate> tag.

In [2]:
# Descargar los XMLs y guardarlos localmente
import requests
import os
import xml.etree.ElementTree as ET
import json
import re
from datetime import datetime

In [3]:
# URLs de los RSS
rss_urls = [
    'https://fapi-top.prisasd.com/podcast/playser/cualquier_tiempo_pasado_fue_anterior/itunestfp/podcast.xml',
    'https://fapi-top.prisasd.com/podcast/playser/todo_concostrina/itunestfp/podcast.xml'
]

# Crear carpeta data si no existe
os.makedirs('../data', exist_ok=True)

# Descargar cada XML
for i, rss_url in enumerate(rss_urls):
    filename = f'feed_{i+1}.xml'
    filepath = os.path.join('../data', filename)
    
    print(f"Descargando {rss_url} -> {filename}")
    response = requests.get(rss_url)
    response.raise_for_status()
    
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(response.text)
    
    print(f"Guardado en {filepath}")

print("XMLs descargados correctamente")

Descargando https://fapi-top.prisasd.com/podcast/playser/cualquier_tiempo_pasado_fue_anterior/itunestfp/podcast.xml -> feed_1.xml
Guardado en ../data/feed_1.xml
Descargando https://fapi-top.prisasd.com/podcast/playser/todo_concostrina/itunestfp/podcast.xml -> feed_2.xml
Guardado en ../data/feed_2.xml
XMLs descargados correctamente


In [4]:
def format_pubdate(pubdate):
    try:
        dt = datetime.strptime(pubdate[:25], '%a, %d %b %Y %H:%M:%S')
        return dt.strftime('%Y_%m_%d_%H')
    except Exception as e:
        print(f'Error formateando fecha: {pubdate} -> {e}')
        return pubdate.replace(' ', '_').replace(':', '_')

def clean_html(raw_html):
    cleanr = re.compile('<.*?>')
    return re.sub(cleanr, '', raw_html)

In [5]:
# Archivos XML locales
xml_files = [
    '../data/feed_1.xml',
    '../data/feed_2.xml'
]

episodes = []

for i, xml_file in enumerate(xml_files):
    print(f"Procesando archivo {i+1}: {xml_file}")
    
    with open(xml_file, 'r', encoding='utf-8') as f:
        xml_content = f.read()
    
    root = ET.fromstring(xml_content)
    feed_episodes = 0
    
    for item in root.findall('.//item'):
        title = item.find('title').text if item.find('title') is not None else ''
        description = item.find('description').text if item.find('description') is not None else ''
        pub_date = item.find('pubDate').text if item.find('pubDate') is not None else ''
        enclosure = item.find('enclosure')
        
        link = enclosure.attrib['url'] if enclosure is not None and 'url' in enclosure.attrib else ''

        description_clean = clean_html(description)
        episodes.append({
            'title': title,
            'description': description_clean,
            'pubDate': pub_date,
            'link': link,
            'audioFileName': format_pubdate(pub_date) + '.mp3'
        })
        feed_episodes += 1
    
    print(f"Feed {i+1} contribuyó con {feed_episodes} episodios")

print(f"Se encontraron {len(episodes)} episodios en total")

# Guardar en JSON
with open('../data/episodes.json', 'w', encoding='utf-8') as f:
    json.dump(episodes, f, ensure_ascii=False, indent=2)

print("Episodios guardados en ../data/episodes.json")

# Mostrar los primeros 3 episodios como ejemplo
print("\nPrimeros 3 episodios:")
for i, episode in enumerate(episodes[:3]):
    print(f"\n{i+1}. {episode['title']}")
    print(f"   Fecha: {episode['pubDate']}")
    print(f"   Descripción: {episode['description'][:100]}...")


Procesando archivo 1: ../data/feed_1.xml
Feed 1 contribuyó con 600 episodios
Procesando archivo 2: ../data/feed_2.xml
Feed 2 contribuyó con 90 episodios
Se encontraron 690 episodios en total
Episodios guardados en ../data/episodes.json

Primeros 3 episodios:

1. Acontece que no es poco | Trabajo infantil, el gran éxito de la Revolución Industrial
   Fecha: Wed, 18 Jun 2025 19:20:00 +0200
   Descripción: Nieves Concostrina habla de la muerte de centenares de niños durante la Revolución Industrial tras t...

2. Acontece que no es poco | La Ley de Amnistía de la Transición o cómo la mafia legitimó el crimen perfecto
   Fecha: Mon, 16 Jun 2025 19:20:00 +0200
   Descripción: Nieves Concostrina habla de la primera propuesta para una Ley de Amnistía registrada por el Partido ...

3. Acontece que no es poco | La fosa de Nigüelas: dos maestros, un guardia civil, un campesino y un cartero
   Fecha: Tue, 17 Jun 2025 19:20:00 +0200
   Descripción: Nieves Concostrina habla sobre la entrega de los r

In [6]:


# Cargar los episodios
with open('../data/episodes.json', 'r', encoding='utf-8') as f:
    episodes = json.load(f)



for ep in episodes:
    url = ep.get('link', '')
    pubdate = ep.get('pubDate', '')
    if not url:
        continue
    filename = format_pubdate(pubdate) + '.mp3'
    filepath = os.path.join('../audios', filename)
    if os.path.exists(filepath):
        print(f'Saltando {filename}, ya existe.')
        continue
    try:
        print(f'Descargando {filename}...')
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(filepath, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f'Guardado en {filepath}')
    except Exception as e:
        print(f'Error descargando {url}: {e}')

Descargando 2025_06_18_19.mp3...
Guardado en ../audios/2025_06_18_19.mp3
Descargando 2025_06_16_19.mp3...
Guardado en ../audios/2025_06_16_19.mp3
Descargando 2025_06_17_19.mp3...
Guardado en ../audios/2025_06_17_19.mp3
Descargando 2025_06_11_19.mp3...
Guardado en ../audios/2025_06_11_19.mp3
Descargando 2025_06_04_19.mp3...
Guardado en ../audios/2025_06_04_19.mp3
Descargando 2025_06_09_19.mp3...
Guardado en ../audios/2025_06_09_19.mp3
Saltando 2025_06_04_19.mp3, ya existe.
Descargando 2025_06_05_19.mp3...
Guardado en ../audios/2025_06_05_19.mp3
Descargando 2025_06_03_19.mp3...
Guardado en ../audios/2025_06_03_19.mp3
Descargando 2025_06_02_19.mp3...
Guardado en ../audios/2025_06_02_19.mp3
Descargando 2025_05_29_19.mp3...
Guardado en ../audios/2025_05_29_19.mp3
Descargando 2025_05_28_19.mp3...
Guardado en ../audios/2025_05_28_19.mp3
Descargando 2025_05_27_19.mp3...
Guardado en ../audios/2025_05_27_19.mp3
Descargando 2025_05_26_19.mp3...
Guardado en ../audios/2025_05_26_19.mp3
Descargando 